# Collaborative Filtering on Netflix Dataset(50 points)

In this part, collaborative filtering on the Netflix prize dataset is implemented

Algorithm from the paper [Empirical Analysis of Predictive Algorithms for Collaborative Filtering](https://arxiv.org/pdf/1301.7363.pdf) is used

## 2.1 Load Netflix Data

The dataset is subset of movie ratings data from the Netflix Prize Challenge. Download the dataset from Piazza. It contains a train set, test set, movie file, and README file. The last two files are original ones from the Netflix Prize, however; in this homework you will deal with train and test files which both are subsets of the Netflix training data. Each of train and test files has lines having this format: MovieID,UserID,Rating.

Your job is to predict a rating in the test set using those provided in the training set.

In [2]:
# load the data, then print out the number of ratings, 
# movies and users in each of train and test sets.
# Your Code Here...
import pandas as pd
import numpy as np

train_data = pd.read_csv("netflix-dataset/TrainingRatings.txt", sep=",", header=None, names = ('movie', 'user', 'rating'))
test_data = pd.read_csv("netflix-dataset/TestingRatings.txt", sep=",", header=None, names = ('movie', 'user', 'rating'))

print "Trainset: Unique movies =", train_data.groupby(['movie']).ngroups, "and unique users =", train_data.groupby(['user']).ngroups, "and total ratings =", train_data['rating'].count()
print "Testset: Unique movies =", test_data.groupby(['movie']).ngroups,"and unique users =", test_data.groupby(['user']).ngroups, "and total ratings =", test_data['rating'].count()

Trainset: Unique movies = 1821 and unique users = 28978 and total ratings = 3255352
Testset: Unique movies = 1701 and unique users = 27555 and total ratings = 100478


In [4]:
# Function to get the user movie pivot table for trainset
def GetPivotTable(file_handle):
    user_movie_array = []
    
    # Parsing the values in the file and appending to user_movie_array
    for line in file_handle:
        values = []
        split_arr = line.rstrip().split(",")
        user_movie_array.append([int(split_arr[1]), int(split_arr[0]), float(split_arr[2])])
        
    # Converting to numpy type for optimization
    user_movie_array = np.array(user_movie_array)
    
    # rows contain unique rows in increasing order. row_pos contains,
    # for each element in the original array whats the correspngind position in the new aray
    # For ex: 1744889 corresponds to 9, so => 9th position in the matrix. So, if we want to extract user for 1st row,
    # get the value from the first element of the row and then index into row to find the user.
    rows, row_pos = np.unique(user_movie_array[:, 0], return_inverse=True)
    #print rows, row_pos, len(rows), len(row_pos)
    
    cols, col_pos = np.unique(user_movie_array[:, 1], return_inverse=True)
    
    # Creating a pivot table type structure which is in a form of a matrix
    pivot_table = np.zeros((len(rows), len(cols)), dtype='float')
    pivot_table[row_pos, col_pos] = user_movie_array[:, 2]

    return pivot_table, rows, row_pos, cols, col_pos

# Function to get the user movie pivot table for testset, structureally same as above function but breaks after 5000 unique users
def GetPivotTableTest(file_handle):
    user_movie_array = []
    
    # Creating a set to get first 5000 unique users
    user_set = set()
    
    # Creating a set to get first 5000 unique users
    user_set = set()
    
    
    for line in file_handle:
        values = []
        split_arr = line.rstrip().split(",")
        if int(split_arr[1]) not in user_set:
            user_set.add(int(split_arr[1]))
        user_movie_array.append([int(split_arr[1]), int(split_arr[0]), float(split_arr[2])])
            
        if(len(user_set) == 5000):
            break
                

    user_movie_array = np.array(user_movie_array)
    # rows contain unique rows in increasing order. row_pos contains,
    # for each element in the original array whats the correspngind position in the new aray
    # For ex: 1744889 corresponds to 9, so => 9th position in the matrix. So, if we want to extract user for 1st row,
    # get the value from the first element of the row and then index into row to find the user.
    rows, row_pos = np.unique(user_movie_array[:, 0], return_inverse=True)
    #print rows, row_pos, len(rows), len(row_pos)
    
    cols, col_pos = np.unique(user_movie_array[:, 1], return_inverse=True)

    pivot_table = np.zeros((len(rows), len(cols)), dtype='float')
    pivot_table[row_pos, col_pos] = user_movie_array[:, 2]

    return pivot_table, rows, row_pos, cols, col_pos

In [5]:
# Loading the dataset
train_data_f = open("netflix-dataset/TrainingRatings.txt", 'r')
test_data_f = open("netflix-dataset/TestingRatings.txt", 'r')
#test_data_f = open("netflix-dataset/TestingSmall.txt", 'r')

# Creating train test datastructures
user_movie_train_dict = {}
user_train_array = []
movie_train_array = []
user_movie_test_dict = {}
user_test_array = []
movie_test_array = []

# Getting the pivot table and data structures from the functions defined above
user_train_array, user_id_train_row, user_id_train_row_index_pos, movie_train_col, movie_train_col_index_pos  = GetPivotTable(train_data_f)
user_test_actual_array, user_id_test_row, user_id_test_row_index_pos, movie_test_col, movie_test_col_index_pos = GetPivotTableTest(test_data_f)
user_test_predict_array = np.zeros((len(user_id_test_row), len(movie_test_col)), dtype='float')

print user_train_array.shape, user_test_actual_array.shape

(28978L, 1821L) (5000L, 124L)


## 2.2 Implement CF

In this part, you will implement the basic collaborative filtering algorithm described in Section 2.1 of the paper -- that is, focus only on Equations 1 and 2 (where Equation 2 is just the Pearson correlation). You should consider the first 5,000 users with their associated items in the test set. 

Note that you should test the algorithm for a small set of users e.g., 10 users first and then run for 5,000 users. It may take long to run but you won't have memory issues. 

Set k to 0.1. 

In [6]:
# I am taking k as per this formula
# https://en.wikipedia.org/wiki/Collaborative_filtering#Memory-based 
k = 0.0001

# Memoization for mean vote for user i
# Computing the average
Vi = np.true_divide(user_train_array.sum(1),(user_train_array!=0).sum(1))
Vi  = Vi.reshape(len(user_train_array),1)
# Converting to a numpy array
np.array(Vi)
# Printing sample values
print Vi

# Memoization for correlation factor Vij and Vij_square. So, that compuatation is efficient.
Vij = np.copy(user_train_array[: , :])

# Mask for keeping the matrix sparse.
Vij_mask = (Vij == 0) 
# Subtracting the mean
Vij -= Vi
# Sparsing operation
Vij[Vij_mask] = 0
# Square operation
Vij_sqr = np.square(Vij)

# Sum of Vij_square for denominator of equation 2
Vij_sum = np.sum(Vij_sqr, axis=1)
#print Vij_sum

[[3.90384615]
 [3.63095238]
 [3.94366197]
 ...
 [3.7721519 ]
 [3.85185185]
 [2.98076923]]


In [7]:
# Function to get mean rating for a particular user
def GetUserVa(user_id):
    return Vi[np.where(user_id_train_row == user_id)]

# Computing the correlation. I am vectorizing the whole operation for better speed.
def correlation(user_id, Vij, Vij_sum):
    # Current user data
    Va = user_train_array[np.where(user_id_train_row == user_id), :]
    Va = np.array(Va)
    Va = Va.reshape(1, len(movie_train_col))
    
    # Subtracting the mean and keeping the matrix sparse
    Va[Va != 0] -= float(GetUserVa(user_id))
    
    # Numerator of equation 2
    numerator = np.dot(Va, Vij.T)
    
    # First half of the denominator
    Va_sqr = np.square(Va)
    
    # Mask for multiplying only common non-zero entries
    mask = np.logical_or(Va_sqr == 0, Vij_sqr == 0)
    # Broadcasting array for vectorization
    Va_sqr = [Va_sqr]*len(user_id_train_row)
    # Masking the non-common zeros
    Va_sqr = np.ma.masked_array(Va_sqr, mask=mask)
    # Reshaping 
    Va_sqr = Va_sqr.reshape(Vij_sqr.shape)
    # Va_sqr sum for denominator
    Va_sqr_sum = np.sum(Va_sqr, axis=1)
    # Reshaping to make it look similar if it is not already.
    Vij_sum = Vij_sum.reshape(Va_sqr_sum.shape)
    
    # Denominator computaition and numpy array conversiion equation (2) of the paper
    denominator = np.sqrt(Va_sqr_sum*Vij_sum)
    denominator = np.array(denominator)
    
    return np.divide(numerator, denominator, out=np.zeros_like(numerator), where=denominator!=0)


def SumOfProduct(user_id, movie, Vij, Vij_sum):
    # Getting the output of equation (2)
    w_a_i = correlation(user_id, Vij, Vij_sum)
    
    # Computing vij - vi  equation (1) considering nonzero weights only.
    vij_norm = np.copy(user_train_array[:,np.where(movie_train_col == movie)])
    vij_norm = vij_norm.reshape(Vi.shape)
    vij_norm_mask = (vij_norm == 0)
    vij_norm -= Vi
    vij_norm[vij_norm_mask] = 0
    
    # Computing the summation part of equation (1)
    sop  = np.dot(w_a_i, vij_norm)
    
    return sop, w_a_i

In [8]:
done = 0

# Profiling the run-time
import time
tic = time.time()

# Computing the actual ratings by doing the computation prescribed in (1) and (2)
for idx in range(0,len(user_id_test_row)):
    for idy in range(0,len(movie_test_col)):
        # No need to compute for non rated items
        if(user_test_actual_array[idx][idy] != 0):
            # Get mean
            Va_final = GetUserVa(user_id_test_row[idx])
            # Get correlation for k computation and second half of equation (1)
            sop, w_a_i = SumOfProduct(user_id_test_row[idx], movie_test_col[idy], Vij, Vij_sum)
            # Compute k as per https://en.wikipedia.org/wiki/Collaborative_filtering#Memory-based
            k_mult_sop = (1/np.sum(np.abs(w_a_i)))*sop
            # Store the prediction in predict array
            user_test_predict_array[idx][idy] =  Va_final + k_mult_sop
            done += 1
            if (done % 1000 == 0):
                print done 
            
toc = time.time()

print "Time is", toc-tic

#print user_test_predict_array, user_test_actual_array

E:\Softwares\Anaconda2\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in double_scalars
E:\Softwares\Anaconda2\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in multiply


1000
2000
3000
4000
5000
Time is 3795.33100009


## 2.3 Evaluation 

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [9]:
# Mean Absolute Error
print "Mean Absolute Error for first 5000 unique users is"
print np.nansum(np.abs(user_test_actual_array-user_test_predict_array))/np.count_nonzero(user_test_predict_array)

Mean Absolute Error for first 5000 unique users is
0.7591577735646583


In [10]:
# Root Mean Squared Error
print "Root Mean Squared Error for first 5000 unique users is"
print np.sqrt(np.nansum(np.square(user_test_actual_array-user_test_predict_array))/np.count_nonzero(user_test_predict_array))

Root Mean Squared Error for first 5000 unique users is
0.9609789112221285


## 2.4 Extensions



In [2]:
# Installing surprise package for the svd implementation
!pip install scikit-surprise

You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [169]:
import pandas as pd
from surprise import Reader, Dataset, dataset, SVD, SVDpp, NMF,  evaluate, accuracy

# Reading the Training ratings
df1 = pd.read_csv("netflix-dataset/TrainingRatings.txt", sep=",", header=None, names = ('itemID', 'userID', 'rating'))

# Using the reader object to convert the pandas dataframe to surprise frame
reader = Reader()
data = Dataset.load_from_df(df1[['userID', 'itemID', 'rating']], reader)

# Using SVD: matrix factorization approach 
algo = SVD(n_epochs = 25, verbose = False)

# Building the raw trainset from the converted pandas frame
trainset = data.build_full_trainset() 

# Training the SVD 
algo.fit(trainset)

In [170]:
# Loading the test set as pandas frame
df2 = pd.read_csv("netflix-dataset/TestingRatings.txt", sep=",", header=None, names = ('itemID', 'userID', 'rating'))

# Removing the duplicate users so that only first 5000 users are picked
df2 = df2.loc[~df2['userID'].duplicated()]

# Selecting first 5000 user specific data frame
df2 = df2.iloc[:5000,:]

# Converting the pandas data frame 
test_data = Dataset.load_from_df(df2[['userID', 'itemID', 'rating']], reader)

# Building the full testset
testset_buf = test_data.build_full_trainset() 
# Converting the testset to test specific format ie masking the ratings so that its reserved for RMSE/MAE computation
testset = testset_buf.build_testset()

# Running the predict method to get the true ratings.
predictions = algo.test(testset)
# Passing the predictions returned by the test method to RMSE and MAE computation functions
print "Mean Absolute Error for first 5000 unique users using SVD is: "
accuracy.mae(predictions, verbose=True)
print "Root Mean Squared Error for first 5000 unique users using SVD is: "
accuracy.rmse(predictions, verbose=True)


Mean Absolute Error for first 5000 unique users using SVD is: 
MAE:  0.6614
Root Mean Squared Error for first 5000 unique users using SVD is: 
RMSE: 0.8542


0.8541781352993629

## SVD for RMSE/MAE improvement

1. I am using the famous SVD algorithm, as popularized by Simon Funk during the Netflix Prize.
https://datajobs.com/data-science-repo/Recommender-Systems-%5BNetflix%5D.pdf

2. Since the matrix factorization technique for sparse matrix is an optimization problem I am using prebuild optimization solvers provided by the surprise package.
3. I am using SGD solver of SVD package and keeping the number of epochs as 25 and number of factors also as 20
4. Learning rates are set to 0.005 and regularization terms are set to 0.02.
5. I also tried some enhancements in the https://arxiv.org/pdf/1301.7363.pdf paper. But the results were not that great. I had also tried NMF approach mentioned here https://ieeexplore.ieee.org/document/6748996/, but the results were not that great. 